In [3]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import pandas as pd
import shoe
import blackjack
import player

In [9]:
#DEPRICATED
#cards DICTIONARY IS NOW STORED IN blackjack.py
#Dictionary of Blackjack Cards
cards = {'two':2,
        'three':3,
        'four':4,
        'five':5,
        'six':6,
        'seven':7,
        'eight':8,
        'nine':9,
        'ten':10,
        'jack':10,
        'queen':10,
        'king':10,
        'ace':[11,1]}

In [8]:
#DEPRICATED
#CreateDeck METHOD IS NOW IN blackjack.py
def CreateDeckOLD():
    #RETURNS: deck - list of str of cards making a standard 52 card deck
    deck = list(cards.keys())*4
#     random.shuffle(deck) #inplace shuffle
    
    return deck

In [21]:
print(f'2 deck cut boundaries: {10/(52*2)} to {15/(52*2)}')
print(f'6 deck cut boundaries: {52/(52*6)} to {78/(52*6)}')
print(f'Dealer always penetrates at a range of 1 to 1.5 decks from the back, or {52/(52*6)} to {78/(52*6)}')

2 deck cut boundaries: 0.09615384615384616 to 0.14423076923076922
6 deck cut boundaries: 0.16666666666666666 to 0.25
Dealer always penetrates at a range of 1 to 1.5 decks from the back, or 0.16666666666666666 to 0.25


In [18]:
#DEPRICATED
#UPDATED VERSION IN shoe.py

def CreateShoeOLD(deck,shoe_size = 6):
    '''
    PARAMETERS:
    shoe_size - int of number of decks in shoe

    RETURNS:
    tuple - list of [shoe_size * deck] cards minus random penetration
            and
            list of cards equal to penetration (reserve cards for dealing last hand)
    '''

    if shoe_size > 2:
        pen_range = [(52/(52*shoe_size)), ((52*1.5)/(52*shoe_size))]
    else:
        #single and double deck penetrate from 10 to 15 cards
        pen_range = [10/(52*shoe_size), 15/(52*shoe_size)]

    #ensures that each deck added to the shoe is shuffled
    #this might just be gamblers superstition on my part...
    shoe = []
    for _ in range(shoe_size):
        random.shuffle(deck)
        shoe += deck
        
    random.shuffle(shoe) #in-place shuffle
    
    #generates penetration at a random specific card based on pen_range
    penetration = round(len(shoe)*random.uniform(pen_range[0],pen_range[1]))
    #generates playbale shoe with cards equal to penetration taken out
    
    return shoe[:-penetration],shoe[-penetration:]

In [4]:
deck = blackjack.CreateDeck()
# type(deck)

In [2]:
playable_shoe = shoe.Shoe()
playable_shoe.CreateShoe(deck)

NameError: name 'deck' is not defined

In [31]:
#Debug purposes only
print(len(playable_shoe._shoe))
card = playable_shoe.DrawCard()
print(card)
print(len(playable_shoe._shoe))

239
eight
238


In [76]:
#DEPRICATED
#GetHandValue IS NOW A METHOD IN blackjack.py
def GetHandValueOLD(hand):
    '''
    Gets the value of a single blackjack hand
    
    PARAMETERS:
    hand - list of cards as str corresponding with cards dictionary
    
    RETURNS:
    hand_value - int in hand value (highest possible hand value if aces are present)
    '''
    
    
    hand_value = 0
    num_aces = hand.count('ace') #gets number of aces
    aces_value = num_aces #assumes all aces are 1's for initial calculation
    
    hand.sort(reverse=True) #sorts all aces to back of array
    for card in hand[:len(hand)-num_aces]: #iterates through list without aces
        hand_value += cards[card] #calculates hand value pre-aces
    
    for i in range(num_aces+1): #iterates +1 in case there is only 1 ace
        if hand_value + ((i*11) + (num_aces-i)) <= 21: #checks combinations of 11s and 1s
            aces_value = (i*11) + (num_aces-i) #sets combination of 11s and 1s if not a bust
        else: #breaks out of loop once the maximum 11s:1s ratio is found
            break
        
    hand_value += aces_value
    
    if hand_value > 21:
        return -1 #-1 is read as a bust when returned
    else:
        return hand_value

In [20]:
def HandAction(shoe_obj,current_player):
    '''
    Case statement of blackjack plays.
    1=Stand, 2=Hit, 3=Double Down, 4= Split, 5=Surrender
    
    PARAMETERS:
    shoe_obj = current shoe being drawn from
    current_player = Player object of current player containing hand(s) and bankroll
    '''
    
    for hand in current_player.GetHands():        
        #flag to tell when to stop prompting for current hand
        acting = True
        while acting:
            
            #assumes you're splitting, every other option turns this off
            splitting = True
            while splitting:
                
                if len(current_player.GetHands()) > 1:
                    for i,hand_to_display in enumerate(current_player.GetHands()):
                        print(f"Split Hand {i}: {hand_to_display['cards']}\nSplit Hand {i} value: {hand_to_display['value']}")

                    print(f"\nCurrent Split Hand is: {hand['cards']}\nCurrent Hand value: {hand['value']}")

                else:
                    print(f"\nCurrent Hand: {hand['cards']}\nCurrent Hand value: {hand['value']}")
                
                print('What would you like to do?')
                _input = input('1 = Stand\n2 = Hit\n3 = Double Down\n4 = Split\n5 = Surrender\n')

                if _input == '1':
                    #stand
                    print('Stand')
                    acting = False
                    splitting = False

                elif _input == '2':
                    #hit
                    print('Hit')
                    current_player.AddCard(hand,shoe_obj.DrawCard())
                    acting = True
                    splitting = False

                elif _input == '3':
                    print('Double Down')
                    #double down
                    #add below if block when money is implemented
                    if current_player.bankroll < hand['bet']*2:
                        print('Not enough money to double down')
                    else:
                        hand['bet'] *= 2
                        current_player.AddCard(hand,shoe_obj.DrawCard())
                        print(f"\nCurrent Hand: {hand['cards']}\nCurrent Hand value: {hand['value']}")
                        acting = False
                        splitting = False

                elif _input == '4':
                    #split
                    print('Split')
                    if len(hand['cards']) > 2 or blackjack.cards[hand['cards'][0]] != blackjack.cards[hand['cards'][1]]: #REMOVE blackjack. WHEN IN blackjack.py
                        print('You can only split pairs. Select another option.')
                    elif current_player.bankroll < hand['bet'] * 2:
                        print('Not enough money to split')
            
                    elif len(current_player.hands) == 4:
                        print('You can only split 3 times.')
                        
                    else:
                        #splits hand into two hands, but WON'T interate to the next one until splitting = False
                        current_player.AddHand([hand['cards'][0],shoe_obj.DrawCard()],hand['bet']) #adding the hand['bet'] effectively doubles the bet
                        hand['cards'] = [hand['cards'][1]] #since only ['cards'] and ['value'] gets edited in the original hand, ['bet'] is retained
                        current_player.AddCard(hand,shoe_obj.DrawCard())

                elif _input == '5':
                    #surrender
                    hand['bet'] = hand['bet']/2
                    hand['value'] = 22
                    print('Surrender (hand value will be set to 22)')
                    
                    acting = False
                    splitting = False

                else:
                    #invalid input
                    print('Please select a valid input.')
                    splitting = False

                

                if hand['value'] > 21:
                    print(f"\nCurrent Hand: {hand['cards']}\nCurrent Hand value: {hand['value']}")
                    print('Bust.')
                    acting = False
                    splitting = False
                elif hand['value'] == 21: #automatically stops if value is 21
                    print('Value is 21.')
                    acting = False
                    splitting = False


In [21]:
def PlayBlackjack():
    '''
    Plays blackjack
    
    No parameters or returns. This is a self-contained method
    that plays blackjack with while loops for user input and dealer decisions.
    '''
    
    #verbose statement
    print('Creating shoe...')
    
    #####################################################################
    deck = blackjack.CreateDeck() #Remove blackjack. once copied into blackjack.py
    #####################################################################
    shoe_obj = shoe.Shoe()
    
    #creates player object with bankroll
    bankroll = 5000
    player_obj = player.Player(bankroll)
    dealer_obj = player.Player()
    
    playing = True
    while playing:
        #checks to see if shoe in the shoe class is empty
        #if empty, it will create a new shoe
        ### ONLY UNCOMMENT WHEN LOOP ISN'T INFINITE ###
        if len(shoe_obj._shoe) == 0:
            #verbose statement
            print('Creating new shoe...')
            shoe_obj.CreateShoe(deck)
        
        #verbose statement
        print('New hand start!')
        #removes all hands from previous rounds
        player_obj.ResetHands()
        dealer_obj.ResetHands()
        
        print('Initial draw...')
        
        player_obj.AddHand([shoe_obj.DrawCard(),shoe_obj.DrawCard()],5)
        dealer_obj.AddHand([shoe_obj.DrawCard(),shoe_obj.DrawCard()],5)
        
        p_hand = player_obj.GetHands()[0] #gets 0th hand because we only have 1 hand right now
        d_hand = dealer_obj.GetHands()[0]

        print(f"Players hand: {p_hand['cards']}\nPlayers hand value: {p_hand['value']}")
        print(f"Dealers hand: {[d_hand['cards'][0]]}\nDealers hand value: {d_hand['shown_value']}")    

        player_acting = True #for while loop below if block
        '''
        Below is a huge block of if statements that are checking
        for player and dealer blackjack cases.
        '''
        if p_hand['value'] == 21:
            print('Blackjack!')
            
            if d_hand['shown_value'] == 11:
                even_money = input('Take even money? (y/n)\n')
                if even_money == 'y':
                    print(f"Player wins {p_hand['bet']}!")
                    player_obj.bankroll += _hand['bet']
                    player_obj.ResetHands()
                    player_acting = False
                else:
                    print("We're playing!")
                    if d_hand['value'] == 21:
                        print("Dealer also has blackjack.\nPlayer pushes.")
                        player_obj.ResetHands()
                        player_acting = False
            else:
                print(f"Player wins {p_hand['bet']*1.5}!")
                player_obj.bankroll += (_hand['bet']*1.5)
                player_obj.ResetHands()
                player_acting = False
            
        if d_hand['shown_value'] == 11 and p_hand['value'] != 21:
            insurance = input('Insurance? (y/n) ')
            
            if d_hand['value'] == 21:
                print(f'Dealer has blackjack with {d_hand["cards"]}')
                
                if insurance == 'y':
                    print("Player has insurance. Bet is lost, but insurance covers it.")
                    #since insurance is effectively a push, nothing happens.
                    player_obj.ResetHands()
                else:
                    print("Player does not have insurance.")
                    player_obj.bankroll -= _hand['bet']
                    player_obj.ResetHands()
                    
                player_acting = False
                
            else:
                print('Dealer does not have blackjack!')
                player_acting = True
        '''
        End of blackjack-checking if statements.
        '''

        '''
        Player decisions
        '''
        if player_acting:
            HandAction(shoe_obj,player_obj)
        
        '''
        Dealer decisions
        '''
        #if player busted out all their hands, dealer doesn't play
        #this for loop checks for any unbusted player hands
        #if blackjack/insurance/dealer blackjack has occurred, player_obj.GetHands() returns an empty list
        dealer_acting = False
        for _hand in player_obj.GetHands():
            if player_acting and _hand['value'] < 22:
                dealer_acting = True
                ##############################
                #COUNT DEALERS DOWN CARD HERE
                ##############################
            
        while dealer_acting:
            print(f"Dealers hand: {d_hand['cards']}\nDealers hand value: {d_hand['value']}")
            
            d_hand = dealer_obj.GetHands()[0]
            if d_hand['value'] == 17 and 'ace' in d_hand['cards']:
                #hit soft 17
                print('Dealer hits soft 17')
                dealer_obj.AddCard(d_hand,shoe_obj.DrawCard())
                
            elif d_hand['value'] < 17:
                #hit
                print('Dealer Hits')
                dealer_obj.AddCard(d_hand,shoe_obj.DrawCard())
                
            else:
                #stand
                print(f"Dealers hand: {d_hand['cards']}\nDealers hand value: {d_hand['value']}")
                print('Dealer Stands')
                dealer_acting = False
        
            if d_hand['value'] > 21:
                print(f"Dealers hand: {d_hand['cards']}\nDealers hand value: {d_hand['value']}")
                print('Dealer bust!')
                dealer_acting = False
        
        '''
        Payout block
        '''
        #if blackjack/insurance/dealer blackjack has occurred, player_obj.GetHands() returns an empty list
        for _hand in player_obj.GetHands():
            if _hand['value'] < 22 and d_hand['value'] > 21:
                print(f"Player has {_hand['cards']} ({_hand['value']}).")
                print(f"Player wins ${_hand['bet']} from dealer bust!")
                player_obj.bankroll += _hand['bet']

            elif _hand['value'] < 22 and _hand['value'] > d_hand['value']:
                print(f"{_hand['cards']} ({_hand['value']}) wins!")
                print(f"Player wins ${_hand['bet']}!")
                player_obj.bankroll += _hand['bet']

            elif _hand['value'] == d_hand['value']:
                print(f"Player's {_hand['cards']} ({_hand['value']}) pushes with Dealer's {d_hand['cards']} ({d_hand['value']}).")
            
            else:
                print(f"Player loses ${_hand['bet']}")
                player_obj.bankroll -= _hand['bet']
            
        if player_obj.bankroll < 1:
            print('OUT OF MONEY')
            playing = False        
        elif input('Continue playing? (y/n)') == 'n':
            playing = False
    

In [22]:
PlayBlackjack()

Creating shoe...
Creating new shoe...
Shuffling 6 decks into shoe...
Cutting...
Burning first card...
Shoe is read to play!
New hand start!
Initial draw...
Players hand: ['king', 'eight']
Players hand value: 18
Dealers hand: ['five']
Dealers hand value: 5

Current Hand: ['king', 'eight']
Current Hand value: 18
What would you like to do?


1 = Stand
2 = Hit
3 = Double Down
4 = Split
5 = Surrender
 2


Hit

Current Hand: ['king', 'eight', 'king']
Current Hand value: 28
Bust.
Player loses $5


Continue playing? (y/n) n
